## 场景描述：
用过往1年的数据训练专家系统，目的是判断用户3个月内是否会离网，本数据为已知是否离网的用户。作为训练和验证数据集。

字段（单位）|英文名称|类型|备注
------------|-------|-----|----
唯一标识|user_id|string|索引
年龄|age|int	
入网时长（月）|net_age|	string	
月均网间通话时长(分）|neto_dur | 	int	
语音通话计费时长（分）|	chrg_dur	|long	
短信计费量（条）|	sms_bill_cnt| 	int	
流量费用（分）	|flux_fee 	|long	
套餐内流量（M)	|iset_flux	|doubel	
月收入（分）	|arpu|	long	|
余额（分）	|acct_balance 	|doubel	|
过缴费期天数（天）|	extend_paydate_days	|int	|
近1年历史欠费次数（次）|	last_year_owe_cnt |int	
近一年投诉次数（次）	|last_year_complain_cnt	|int	
已订购电信业务数量（次）|	ordered_product_cnt|	int	
稳定社交圈成员数量（个）|	circle_nou|	int	
是否离网（1/0）		|churn_label |int|	标签，1表示已离网，0表示未离网


In [3]:
#!-*- coding:utf-8 -*-
#导入常用的库
# numpy: python中常用的多维数值计算库
# pandas:python中的数据分析库
# matplotlib :python中的画图库
# sklearn: 常用的机器学期库
    
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn import cross_validation

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
#导入数据
data=pd.read_csv('decision.csv',encoding='utf-8',sep=' ')

In [5]:
#打印数据的前5行
data.head(5)

,user_id,age,net_age,neto_dur,chrg_dur,sms_bill_cnt,flux_fee,iset_flux,arpu,acct_balance,extend_paydate_days,last_year_owe_cnt,last_year_complain_cnt,ordered_prodect_cnt,circle_nou,churn_label
0,8408056510,31,25,42,42,16,2717,10240,21383,4162,30,0,0,3,4,0
1,8627971378,37,23,97,42,21,2382,1024,7847,5884,47,5,0,1,4,0
2,8707700881,35,20,73,27,4,2152,300,1941,3358,40,0,0,4,2,1
3,8404003205,42,15,97,27,21,2024,200,2656,2487,70,0,0,1,6,0
4,8701397825,39,21,71,8,15,2336,10240,9954,3052,70,3,3,2,6,0


In [6]:
# 显示数据的形状
print(data.shape)

(1000000, 16)


In [7]:
#显示整个表格的信息
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 16 columns):
user_id                   1000000 non-null int64
age                       1000000 non-null int64
net_age                   1000000 non-null int64
neto_dur                  1000000 non-null int64
chrg_dur                  1000000 non-null int64
sms_bill_cnt              1000000 non-null int64
flux_fee                  1000000 non-null int64
iset_flux                 1000000 non-null int64
arpu                      1000000 non-null int64
acct_balance              1000000 non-null int64
extend_paydate_days       1000000 non-null int64
last_year_owe_cnt         1000000 non-null int64
last_year_complain_cnt    1000000 non-null int64
ordered_prodect_cnt       1000000 non-null int64
circle_nou                1000000 non-null int64
churn_label               1000000 non-null int64
dtypes: int64(16)
memory usage: 122.1 MB


In [8]:
data.describe()

,user_id,age,net_age,neto_dur,chrg_dur,sms_bill_cnt,flux_fee,iset_flux,arpu,acct_balance,extend_paydate_days,last_year_owe_cnt,last_year_complain_cnt,ordered_prodect_cnt,circle_nou,churn_label
count,1.000000e+06,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,8.499994e+09,35.027698,30.326975,59.574035,29.548803,15.079246,2000.137351,2787.907616,7724.929542,4109.592104,39.953382,0.656272,0.655834,3.501180,3.884833,0.500786
std,2.887864e+08,11.340514,14.144417,19.880181,9.943020,7.429960,598.237676,3094.369936,5907.102323,1881.612053,23.379563,1.313058,1.312611,1.707307,1.949743,0.500000
min,8.000001e+09,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,200.000000,-12655.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,8.249915e+09,26.000000,20.000000,46.000000,23.000000,10.000000,1596.000000,500.000000,3198.000000,2758.000000,20.000000,0.000000,0.000000,2.000000,2.000000,0.000000
50%,8.499927e+09,34.000000,30.000000,60.000000,30.000000,15.000000,2000.000000,1536.000000,5915.000000,4055.000000,40.000000,0.000000,0.000000,4.000000,4.000000,1.000000
75%,8.750455e+09,42.000000,40.000000,73.000000,36.000000,20.000000,2404.000000,4096.000000,10643.000000,5383.000000,60.000000,1.000000,0.000000,5.000000,5.000000,1.000000
max,9.000000e+09,101.000000,107.000000,158.000000,77.000000,55.000000,4897.000000,10240.000000,48431.000000,14183.000000,80.000000,5.000000,5.000000,6.000000,7.000000,1.000000


In [9]:
col_dicts = {}
cols =  data.columns.values.tolist()

X=data.loc[:,cols[1:-1]]
# print X.info()
y=data[cols[-1]]

In [17]:
len(cols)

16

In [10]:
X.head()

,age,net_age,neto_dur,chrg_dur,sms_bill_cnt,flux_fee,iset_flux,arpu,acct_balance,extend_paydate_days,last_year_owe_cnt,last_year_complain_cnt,ordered_prodect_cnt,circle_nou
0,31,25,42,42,16,2717,10240,21383,4162,30,0,0,3,4
1,37,23,97,42,21,2382,1024,7847,5884,47,5,0,1,4
2,35,20,73,27,4,2152,300,1941,3358,40,0,0,4,2
3,42,15,97,27,21,2024,200,2656,2487,70,0,0,1,6
4,39,21,71,8,15,2336,10240,9954,3052,70,3,3,2,6


In [11]:
#分割数据集为训练集和测试集
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.3, random_state=0)

In [13]:
#显示训练集和测试集所占的比例
print(y_train.value_counts()/len(y_train))
print(y_test.value_counts()/len(y_test))

1    0.500871
0    0.499129
Name: churn_label, dtype: float64
1    0.500587
0    0.499413
Name: churn_label, dtype: float64


In [14]:
# 构建随机森林模型
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=10)
print(rf_clf.fit(X_train, y_train))


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [16]:
rf_pred = rf_clf.predict(X_test)
from sklearn import metrics
print(metrics.classification_report(y_test, rf_pred))
print(metrics.confusion_matrix(y_test, rf_pred))
print(metrics.accuracy_score(y_test, rf_pred))

             precision    recall  f1-score   support

          0       0.51      0.62      0.56    149824
          1       0.52      0.40      0.45    150176

avg / total       0.51      0.51      0.51    300000

[[93590 56234]
 [89693 60483]]
0.5135766666666667
